In [1]:
import pandas as pd
import os
import matplotlib.pyplot as plt


In [2]:
# Define algumas variáveis

pasta_datasets = '../Datasets/'
planilha_dolar_ipca = 'dolar_ipca.csv'
planilha_combustiveis = 'combustiveis.csv'

In [3]:
# Efetua a leitura do arquivo 'combustiveis.csv' 
df_combustiveis =  pd.read_csv(pasta_datasets+planilha_combustiveis,sep = ';',low_memory=False, 
                               parse_dates=['Data'],dayfirst = True,encoding='utf-8')

# Cria um novo índice com a coluna 'Data' 
df_combustiveis.set_index(['Data'], inplace=True)

display(df_combustiveis)
df_combustiveis.info()

,Regiao,Estado,Municipio,Produto,ValorVenda
Data,,,,,
2005-01-04,SE,SP,GUARULHOS,GASOLINA,2.257
2005-01-04,SE,SP,GUARULHOS,ETANOL,1.449
2005-01-04,SE,SP,SOROCABA,GASOLINA,2.210
2005-01-04,SE,SP,SOROCABA,ETANOL,1.230
2005-01-03,CO,DF,BRASILIA,GASOLINA,2.150
...,...,...,...,...,...
2021-12-31,SE,RJ,CAMPOS DOS GOYTACAZES,GASOLINA,7.090
2021-12-31,NE,BA,JEQUIE,ETANOL,5.859
2021-12-31,NE,BA,JEQUIE,GASOLINA,6.799


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 12900965 entries, 2005-01-04 to 2021-12-31
Data columns (total 5 columns):
 #   Column      Dtype  
---  ------      -----  
 0   Regiao      object 
 1   Estado      object 
 2   Municipio   object 
 3   Produto     object 
 4   ValorVenda  float64
dtypes: float64(1), object(4)
memory usage: 590.6+ MB


In [4]:
# A partir daqui os dados de venda dos municípios não serão utilizados diretamente, mas será feita a média
# dos valores por combustível no estado num mesmo dia.

In [5]:
# Como a idéia é montar uma tabela agregando numa mesma linha/dia os valores de venda do Etanol e da Gasolina, 
# para facilitar a análise, serão feitas duas tabelas e depois serão agrupadas

In [6]:
# Inicia montando um dataframe para Gasolina, com os valores agrupados inclusive por Estado

df_gasolina_estados = pd.DataFrame(df_combustiveis[df_combustiveis["Produto"] =='GASOLINA'].
                                   groupby(by=['Data','Regiao','Estado','Produto'])
                                   ['ValorVenda'].mean())

display(df_gasolina_estados)

ValorVenda
Data       Regiao Estado Produto             
2005-01-03 CO     DF     GASOLINA    2.252160
                  GO     GASOLINA    2.362289
                  MS     GASOLINA    2.542967
                  MT     GASOLINA    2.740508
           N      AC     GASOLINA    2.648780
...                                       ...
2021-12-31 NE     BA     GASOLINA    6.750375
           SE     ES     GASOLINA    6.857250
                  MG     GASOLINA    6.817750
                  RJ     GASOLINA    7.094333
                  SP     GASOLINA    6.150905

[75186 rows x 1 columns]

In [7]:
# Renomeia a coluna de ValorVenda para ValorVendaGasolina, a fim de diferenciar quando fizer o merge

df_gasolina_estados.rename(columns= {'ValorVenda':'ValorVendaGasolina'}, inplace = True)

display(df_gasolina_estados)

df_gasolina_estados.info()

ValorVendaGasolina
Data       Regiao Estado Produto                     
2005-01-03 CO     DF     GASOLINA            2.252160
                  GO     GASOLINA            2.362289
                  MS     GASOLINA            2.542967
                  MT     GASOLINA            2.740508
           N      AC     GASOLINA            2.648780
...                                               ...
2021-12-31 NE     BA     GASOLINA            6.750375
           SE     ES     GASOLINA            6.857250
                  MG     GASOLINA            6.817750
                  RJ     GASOLINA            7.094333
                  SP     GASOLINA            6.150905

[75186 rows x 1 columns]

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 75186 entries, (Timestamp('2005-01-03 00:00:00'), 'CO', 'DF', 'GASOLINA') to (Timestamp('2021-12-31 00:00:00'), 'SE', 'SP', 'GASOLINA')
Data columns (total 1 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   ValorVendaGasolina  75186 non-null  float64
dtypes: float64(1)
memory usage: 983.4+ KB


In [8]:
# A coluna 'Produto' não é mais necessária, e poderá causar problemas ao efetuar o merge com o dataframe do
# Etanol, desta forma ela será removida

# Como esta coluna é parte de um multi-index, inicialmente é necessário resetar o índice desta coluna

df_gasolina_estados = df_gasolina_estados.reset_index(level=['Produto'])

display(df_gasolina_estados)

Produto  ValorVendaGasolina
Data       Regiao Estado                              
2005-01-03 CO     DF      GASOLINA            2.252160
                  GO      GASOLINA            2.362289
                  MS      GASOLINA            2.542967
                  MT      GASOLINA            2.740508
           N      AC      GASOLINA            2.648780
...                            ...                 ...
2021-12-31 NE     BA      GASOLINA            6.750375
           SE     ES      GASOLINA            6.857250
                  MG      GASOLINA            6.817750
                  RJ      GASOLINA            7.094333
                  SP      GASOLINA            6.150905

[75186 rows x 2 columns]

In [9]:
# Agora é possível removê-la
df_gasolina_estados = df_gasolina_estados.drop('Produto',1)

display(df_gasolina_estados)

ValorVendaGasolina
Data       Regiao Estado                    
2005-01-03 CO     DF                2.252160
                  GO                2.362289
                  MS                2.542967
                  MT                2.740508
           N      AC                2.648780
...                                      ...
2021-12-31 NE     BA                6.750375
           SE     ES                6.857250
                  MG                6.817750
                  RJ                7.094333
                  SP                6.150905

[75186 rows x 1 columns]

In [10]:
# Verifica que não existem valores null
df_gasolina_estados.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 75186 entries, (Timestamp('2005-01-03 00:00:00'), 'CO', 'DF') to (Timestamp('2021-12-31 00:00:00'), 'SE', 'SP')
Data columns (total 1 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   ValorVendaGasolina  75186 non-null  float64
dtypes: float64(1)
memory usage: 909.9+ KB


In [11]:
# Da mesma forma, monta o dataframe para Etanol, com os valores agrupados inclusive por Estado

df_etanol_estados = pd.DataFrame(df_combustiveis[df_combustiveis["Produto"] =='ETANOL'].
                                 groupby(by=['Data','Regiao','Estado','Produto'])
                                 ['ValorVenda'].mean())

display(df_etanol_estados)

ValorVenda
Data       Regiao Estado Produto            
2005-01-03 CO     DF     ETANOL     1.696821
                  GO     ETANOL     1.591160
                  MS     ETANOL     1.732000
                  MT     ETANOL     1.767554
           N      AC     ETANOL     1.990000
...                                      ...
2021-12-31 NE     BA     ETANOL     5.738000
           SE     ES     ETANOL     6.146333
                  MG     ETANOL     4.985167
                  RJ     ETANOL     6.056833
                  SP     ETANOL     4.667500

[72726 rows x 1 columns]

In [12]:
# Renomeia a coluna de ValorVenda para ValorVendaEtanol, a fim de diferenciar quando fizer o merge

df_etanol_estados.rename(columns= {'ValorVenda':'ValorVendaEtanol'}, inplace = True)

display(df_etanol_estados)

df_etanol_estados.info()

ValorVendaEtanol
Data       Regiao Estado Produto                  
2005-01-03 CO     DF     ETANOL           1.696821
                  GO     ETANOL           1.591160
                  MS     ETANOL           1.732000
                  MT     ETANOL           1.767554
           N      AC     ETANOL           1.990000
...                                            ...
2021-12-31 NE     BA     ETANOL           5.738000
           SE     ES     ETANOL           6.146333
                  MG     ETANOL           4.985167
                  RJ     ETANOL           6.056833
                  SP     ETANOL           4.667500

[72726 rows x 1 columns]

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 72726 entries, (Timestamp('2005-01-03 00:00:00'), 'CO', 'DF', 'ETANOL') to (Timestamp('2021-12-31 00:00:00'), 'SE', 'SP', 'ETANOL')
Data columns (total 1 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   ValorVendaEtanol  72726 non-null  float64
dtypes: float64(1)
memory usage: 952.1+ KB


In [13]:
# A coluna 'Produto' não é mais necessária, e poderá causar problemas ao efetuar o merge com o dataframe da
# Gasolina, desta forma ela será removida

# Como esta coluna é parte de um multi-index, inicialmente é necessário resetar o índice desta coluna

df_etanol_estados = df_etanol_estados.reset_index(level=['Produto'])

display(df_etanol_estados)

Produto  ValorVendaEtanol
Data       Regiao Estado                          
2005-01-03 CO     DF      ETANOL          1.696821
                  GO      ETANOL          1.591160
                  MS      ETANOL          1.732000
                  MT      ETANOL          1.767554
           N      AC      ETANOL          1.990000
...                          ...               ...
2021-12-31 NE     BA      ETANOL          5.738000
           SE     ES      ETANOL          6.146333
                  MG      ETANOL          4.985167
                  RJ      ETANOL          6.056833
                  SP      ETANOL          4.667500

[72726 rows x 2 columns]

In [14]:
# Agora é possível removê-la
df_etanol_estados = df_etanol_estados.drop('Produto',1)

display(df_etanol_estados)

ValorVendaEtanol
Data       Regiao Estado                  
2005-01-03 CO     DF              1.696821
                  GO              1.591160
                  MS              1.732000
                  MT              1.767554
           N      AC              1.990000
...                                    ...
2021-12-31 NE     BA              5.738000
           SE     ES              6.146333
                  MG              4.985167
                  RJ              6.056833
                  SP              4.667500

[72726 rows x 1 columns]

In [15]:
# Verifica que não existem valores null
df_etanol_estados.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 72726 entries, (Timestamp('2005-01-03 00:00:00'), 'CO', 'DF') to (Timestamp('2021-12-31 00:00:00'), 'SE', 'SP')
Data columns (total 1 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   ValorVendaEtanol  72726 non-null  float64
dtypes: float64(1)
memory usage: 881.1+ KB


In [16]:
# Importante observar que havia 146238 linhas anteriormente, sendo 74334 relativas a Gasolina e 71904 a Etanol.

In [17]:
# Com os dataframes prontos, de Gasolina e Etanol, será feito o merge deles
df_combustiveis_agrupados_estado = pd.merge(df_gasolina_estados,df_etanol_estados,
                                            on=["Data","Regiao","Estado"],how="outer")

display(df_combustiveis_agrupados_estado)


ValorVendaGasolina  ValorVendaEtanol
Data       Regiao Estado                                      
2005-01-03 CO     DF                2.252160          1.696821
                  GO                2.362289          1.591160
                  MS                2.542967          1.732000
                  MT                2.740508          1.767554
           N      AC                2.648780          1.990000
...                                      ...               ...
2019-11-27 NE     RN                     NaN          3.670000
2019-12-19 NE     BA                     NaN          3.389000
2020-05-07 CO     GO                     NaN          2.999000
2020-11-19 SE     ES                     NaN          3.359000
2021-10-14 NE     BA                     NaN          4.999000

[75211 rows x 2 columns]

In [18]:
# Verifica a existência de valores null
df_combustiveis_agrupados_estado.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 75211 entries, (Timestamp('2005-01-03 00:00:00'), 'CO', 'DF') to (Timestamp('2021-10-14 00:00:00'), 'NE', 'BA')
Data columns (total 2 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   ValorVendaGasolina  75186 non-null  float64
 1   ValorVendaEtanol    72726 non-null  float64
dtypes: float64(2)
memory usage: 1.6+ MB


In [19]:
# Verifica que existem dias nos quais não houve coleta de dados em determinado estado, 
# resultando em valores nulos

qtd_linhas = df_combustiveis_agrupados_estado.shape[0]
qtd_valores_gasolina = sum(pd.notnull(df_combustiveis_agrupados_estado['ValorVendaGasolina']))
qtd_valores_etanol = sum(pd.notnull(df_combustiveis_agrupados_estado['ValorVendaEtanol']))
qtd_valores_nulos_gasolina = sum(pd.isnull(df_combustiveis_agrupados_estado['ValorVendaGasolina']))
valores_nulos_etanol = sum(pd.isnull(df_combustiveis_agrupados_estado['ValorVendaEtanol']))

print('Total de linhas: ' + str(qtd_linhas))
print('Quantidade de valores de gasolina: ' + str(qtd_valores_gasolina))
print('Quantidade de valores nulos de gasolina: ' + str(qtd_valores_nulos_gasolina))
print('Quantidade de valores de etanol: ' + str(qtd_valores_etanol))
print('Quantidade de valores nulos de etanol: ' + str(valores_nulos_etanol))

Total de linhas: 75211
Quantidade de valores de gasolina: 75186
Quantidade de valores nulos de gasolina: 25
Quantidade de valores de etanol: 72726
Quantidade de valores nulos de etanol: 2485


In [20]:
# Optou-se por remover as linhas com valor nulo de combustíveis

df_combustiveis_agrupados_estado.dropna(subset = ['ValorVendaGasolina'], inplace=True)
df_combustiveis_agrupados_estado.dropna(subset = ['ValorVendaEtanol'], inplace=True)

display(df_combustiveis_agrupados_estado)
df_combustiveis_agrupados_estado.info()

ValorVendaGasolina  ValorVendaEtanol
Data       Regiao Estado                                      
2005-01-03 CO     DF                2.252160          1.696821
                  GO                2.362289          1.591160
                  MS                2.542967          1.732000
                  MT                2.740508          1.767554
           N      AC                2.648780          1.990000
...                                      ...               ...
2021-12-31 NE     BA                6.750375          5.738000
           SE     ES                6.857250          6.146333
                  MG                6.817750          4.985167
                  RJ                7.094333          6.056833
                  SP                6.150905          4.667500

[72701 rows x 2 columns]

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 72701 entries, (Timestamp('2005-01-03 00:00:00'), 'CO', 'DF') to (Timestamp('2021-12-31 00:00:00'), 'SE', 'SP')
Data columns (total 2 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   ValorVendaGasolina  72701 non-null  float64
 1   ValorVendaEtanol    72701 non-null  float64
dtypes: float64(2)
memory usage: 1.5+ MB


In [21]:
# Exporta o dataset para um arquivo CSV
#df_combustiveis_agrupados_estado.to_csv(pasta_datasets + 'df_combustiveis_agrupados_estado.csv', sep = ';',index=True)

In [22]:
# Efetua a leitura do arquivo 'dolar_ipca.csv'

print(pasta_datasets+planilha_dolar_ipca)
df_dolar_ipca =  pd.read_csv(pasta_datasets+planilha_dolar_ipca,sep = ';',low_memory=False, parse_dates=['Data'],dayfirst = True,encoding='utf-8')

# Cria um novo índice com a coluna 'Data' 
df_dolar_ipca.set_index(['Data'], inplace=True)

display(df_dolar_ipca)
df_dolar_ipca.info()



../Datasets/dolar_ipca.csv


,cotacaoCompra,cotacaoVenda,Indice_Dez93,Ipca_Mensal
Data,,,,
2005-01-03,2.6674,2.6682,2412.83,0.58
2005-01-04,2.6879,2.6887,2412.83,0.58
2005-01-05,2.7088,2.7096,2412.83,0.58
2005-01-06,2.7199,2.7207,2412.83,0.58
2005-01-07,2.7024,2.7032,2412.83,0.58
...,...,...,...,...
2021-12-27,5.6644,5.6650,6120.04,0.73
2021-12-28,5.6432,5.6438,6120.04,0.73
2021-12-29,5.6613,5.6619,6120.04,0.73


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 4268 entries, 2005-01-03 to 2021-12-31
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   cotacaoCompra  4268 non-null   float64
 1   cotacaoVenda   4268 non-null   float64
 2   Indice_Dez93   4268 non-null   float64
 3   Ipca_Mensal    4268 non-null   float64
dtypes: float64(4)
memory usage: 166.7 KB


In [23]:
# Remove a coluna cotaCompra, referente ao dólar de compra, e que não será utilizada

df_dolar_ipca = df_dolar_ipca.drop(['cotacaoCompra'],axis=1)

In [24]:
# Renomeia a coluna "cotacaoVenda" para "CotacaoDolarVenda"

df_dolar_ipca = df_dolar_ipca.rename(columns={'cotacaoVenda':'CotacaoDolarVenda'})

In [25]:
display(df_combustiveis_agrupados_estado)
df_combustiveis_agrupados_estado.info()

ValorVendaGasolina  ValorVendaEtanol
Data       Regiao Estado                                      
2005-01-03 CO     DF                2.252160          1.696821
                  GO                2.362289          1.591160
                  MS                2.542967          1.732000
                  MT                2.740508          1.767554
           N      AC                2.648780          1.990000
...                                      ...               ...
2021-12-31 NE     BA                6.750375          5.738000
           SE     ES                6.857250          6.146333
                  MG                6.817750          4.985167
                  RJ                7.094333          6.056833
                  SP                6.150905          4.667500

[72701 rows x 2 columns]

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 72701 entries, (Timestamp('2005-01-03 00:00:00'), 'CO', 'DF') to (Timestamp('2021-12-31 00:00:00'), 'SE', 'SP')
Data columns (total 2 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   ValorVendaGasolina  72701 non-null  float64
 1   ValorVendaEtanol    72701 non-null  float64
dtypes: float64(2)
memory usage: 1.5+ MB


In [26]:
# O dataframe df_combustiveis_agrupados_estado possui um multi-index, e antes de efetuar o merge com o 
# dataframe df_dolar_ipca será necessário resetar as colunas 'Região' e 'Estado' daquele, a fim de utilizar 
# apenas a coluna 'Data' como chave

df_combustiveis_agrupados_estado = df_combustiveis_agrupados_estado.reset_index(level=['Regiao','Estado'])

display(df_combustiveis_agrupados_estado)

,Regiao,Estado,ValorVendaGasolina,ValorVendaEtanol
Data,,,,
2005-01-03,CO,DF,2.252160,1.696821
2005-01-03,CO,GO,2.362289,1.591160
2005-01-03,CO,MS,2.542967,1.732000
2005-01-03,CO,MT,2.740508,1.767554
2005-01-03,N,AC,2.648780,1.990000
...,...,...,...,...
2021-12-31,NE,BA,6.750375,5.738000
2021-12-31,SE,ES,6.857250,6.146333
2021-12-31,SE,MG,6.817750,4.985167


In [27]:
df_combustiveis_agrupados_estado.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 72701 entries, 2005-01-03 to 2021-12-31
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Regiao              72701 non-null  object 
 1   Estado              72701 non-null  object 
 2   ValorVendaGasolina  72701 non-null  float64
 3   ValorVendaEtanol    72701 non-null  float64
dtypes: float64(2), object(2)
memory usage: 2.8+ MB


In [28]:
# Efetua o merge com o dataframe df_dolar_ipca

df_combustiveis_dolar_ipca_estado = pd.merge(df_combustiveis_agrupados_estado,
                                             df_dolar_ipca,on="Data",how="left")

display(df_combustiveis_dolar_ipca_estado)


,Regiao,Estado,ValorVendaGasolina,ValorVendaEtanol,CotacaoDolarVenda,Indice_Dez93,Ipca_Mensal
Data,,,,,,,
2005-01-03,CO,DF,2.252160,1.696821,2.6682,2412.83,0.58
2005-01-03,CO,GO,2.362289,1.591160,2.6682,2412.83,0.58
2005-01-03,CO,MS,2.542967,1.732000,2.6682,2412.83,0.58
2005-01-03,CO,MT,2.740508,1.767554,2.6682,2412.83,0.58
2005-01-03,N,AC,2.648780,1.990000,2.6682,2412.83,0.58
...,...,...,...,...,...,...,...
2021-12-31,NE,BA,6.750375,5.738000,5.5805,6120.04,0.73
2021-12-31,SE,ES,6.857250,6.146333,5.5805,6120.04,0.73
2021-12-31,SE,MG,6.817750,4.985167,5.5805,6120.04,0.73


In [29]:
# Verifica a existência de valores null, sendo que:
# - para as colunas derivadas do dataframe df_dolar_ipca, verifica se que há 1753 valores nulos,
# o que será verificado a seguir

df_combustiveis_dolar_ipca_estado.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 72701 entries, 2005-01-03 to 2021-12-31
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Regiao              72701 non-null  object 
 1   Estado              72701 non-null  object 
 2   ValorVendaGasolina  72701 non-null  float64
 3   ValorVendaEtanol    72701 non-null  float64
 4   CotacaoDolarVenda   70948 non-null  float64
 5   Indice_Dez93        70948 non-null  float64
 6   Ipca_Mensal         70948 non-null  float64
dtypes: float64(5), object(2)
memory usage: 4.4+ MB


In [30]:
# Como as colunas cotacaoCompra, cotacaoVenda, Indice_Dez93 e Ipca_Mensal possuem a mesma quantidade de 
# valores nulos, optou-se por utilizar a cotacaoVenda para verificar quais as Datas (index) que não 
# possuem dados

datas_valores_nulos_dolar = df_combustiveis_dolar_ipca_estado[df_combustiveis_dolar_ipca_estado['CotacaoDolarVenda'].isnull()].index
datas_valores_nulos_dolar_distintos = df_combustiveis_dolar_ipca_estado[df_combustiveis_dolar_ipca_estado['CotacaoDolarVenda'].isnull()].index.unique()

print('Quantidade de datas com valores nulos na coluna CotacaoDolarVenda: ' + str(datas_valores_nulos_dolar.size))
print('Quantidade de datas distintas com valores nulos na coluna CotacaoDolarVenda: ' + str(datas_valores_nulos_dolar_distintos.size))

Quantidade de datas com valores nulos na coluna CotacaoDolarVenda: 1753
Quantidade de datas distintas com valores nulos na coluna CotacaoDolarVenda: 123


In [31]:
# Verifica que o dataframe df_dolar_ipca contém apenas dias úteis (dayofweek de 0 a 4)

# 0 = Segunda-feira
# 1 = Terça-feira
# 2 = Quarta-feira
# 3 = Quinta-feira
# 4 = Sexta-feira
# 5 = Sábado
# 6 = Domingo

df_dolar_ipca.index.dayofweek.unique()

Int64Index([0, 1, 2, 3, 4], dtype='int64', name='Data')

In [32]:
# Verifica que há dias em que houve leitura aos finais de semana (dayofweek 5 e 6), o que explicaria a 
# ausência de alguns dados na tabela, já que não existem finais de semana em df_dolar_ipca

datas_valores_nulos_dolar_distintos.dayofweek.unique()

Int64Index([0, 1, 3, 5, 6, 2, 4], dtype='int64', name='Data')

In [33]:
# Faz uma lista com os índices correspondentes aos dias úteis da semana

dias_uteis = [0,1,2,3,4]

In [34]:
# Lista as datas distintas 
datas_valores_nulos_dolar_distintos

DatetimeIndex(['2005-02-07', '2005-02-08', '2005-04-21', '2005-05-26',
               '2005-06-04', '2006-02-27', '2007-02-19', '2008-02-04',
               '2009-02-23', '2009-08-30',
               ...
               '2021-08-01', '2021-08-08', '2021-08-15', '2021-08-22',
               '2021-08-29', '2021-09-05', '2021-09-07', '2021-10-12',
               '2021-11-02', '2021-11-15'],
              dtype='datetime64[ns]', name='Data', length=123, freq=None)

In [35]:
# Percorre a lista removendo as datas que não são finais de semana, de forma a gerar uma lista 
# de datas para análise

offset = 0

datas_uteis_valores_nulos_dolar_distintos = datas_valores_nulos_dolar_distintos

for data in datas_valores_nulos_dolar_distintos:
    if data.dayofweek not in dias_uteis:
        datas_uteis_valores_nulos_dolar_distintos = datas_uteis_valores_nulos_dolar_distintos.delete(offset)
    else:
        offset = offset + 1
    
print('Quantidade de dias úteis: ' + str(datas_uteis_valores_nulos_dolar_distintos.size))

Quantidade de dias úteis: 80


In [36]:
# Confirma que a lista não tem final de semana

datas_uteis_valores_nulos_dolar_distintos.dayofweek.unique()

Int64Index([0, 1, 3, 2, 4], dtype='int64', name='Data')

In [37]:
# Fazendo uma análise nestas datas, conclui-se que, apesar de não serem finais de semana, realmente 
# não foram dias úteis, mas feriados (Carnaval, Tiradentes, Independência, Natal, entre outros)

datas_uteis_valores_nulos_dolar_distintos

DatetimeIndex(['2005-02-07', '2005-02-08', '2005-04-21', '2005-05-26',
               '2006-02-27', '2007-02-19', '2008-02-04', '2009-02-23',
               '2010-02-15', '2010-02-16', '2010-04-21', '2010-09-07',
               '2010-10-12', '2010-11-02', '2010-11-15', '2011-03-07',
               '2011-03-08', '2011-04-21', '2011-06-23', '2011-09-07',
               '2011-10-12', '2011-11-02', '2011-11-15', '2012-02-20',
               '2012-02-21', '2012-05-01', '2013-02-11', '2013-02-12',
               '2013-05-01', '2013-12-25', '2014-01-01', '2014-03-03',
               '2014-03-04', '2014-04-21', '2014-12-25', '2015-01-01',
               '2015-02-16', '2015-02-17', '2015-04-21', '2015-09-07',
               '2015-10-12', '2015-11-02', '2016-02-08', '2016-02-09',
               '2016-09-07', '2016-10-12', '2016-11-02', '2016-11-15',
               '2017-02-27', '2017-02-28', '2017-05-01', '2017-11-15',
               '2017-12-25', '2018-01-01', '2018-02-12', '2018-02-13',
      

In [38]:
# Desta forma, realmente não deveriam existir no dataframe df_dolar_ipca
# Assim será feita a remoção destas linhas que não são dias uteis

In [39]:
# ANTES

display(df_combustiveis_dolar_ipca_estado)
df_combustiveis_dolar_ipca_estado.info()

,Regiao,Estado,ValorVendaGasolina,ValorVendaEtanol,CotacaoDolarVenda,Indice_Dez93,Ipca_Mensal
Data,,,,,,,
2005-01-03,CO,DF,2.252160,1.696821,2.6682,2412.83,0.58
2005-01-03,CO,GO,2.362289,1.591160,2.6682,2412.83,0.58
2005-01-03,CO,MS,2.542967,1.732000,2.6682,2412.83,0.58
2005-01-03,CO,MT,2.740508,1.767554,2.6682,2412.83,0.58
2005-01-03,N,AC,2.648780,1.990000,2.6682,2412.83,0.58
...,...,...,...,...,...,...,...
2021-12-31,NE,BA,6.750375,5.738000,5.5805,6120.04,0.73
2021-12-31,SE,ES,6.857250,6.146333,5.5805,6120.04,0.73
2021-12-31,SE,MG,6.817750,4.985167,5.5805,6120.04,0.73


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 72701 entries, 2005-01-03 to 2021-12-31
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Regiao              72701 non-null  object 
 1   Estado              72701 non-null  object 
 2   ValorVendaGasolina  72701 non-null  float64
 3   ValorVendaEtanol    72701 non-null  float64
 4   CotacaoDolarVenda   70948 non-null  float64
 5   Indice_Dez93        70948 non-null  float64
 6   Ipca_Mensal         70948 non-null  float64
dtypes: float64(5), object(2)
memory usage: 4.4+ MB


In [40]:
# DEPOIS

df_combustiveis_dolar_ipca_estado.dropna(subset = ['CotacaoDolarVenda'], inplace=True)

display(df_combustiveis_dolar_ipca_estado)
df_combustiveis_dolar_ipca_estado.info()

,Regiao,Estado,ValorVendaGasolina,ValorVendaEtanol,CotacaoDolarVenda,Indice_Dez93,Ipca_Mensal
Data,,,,,,,
2005-01-03,CO,DF,2.252160,1.696821,2.6682,2412.83,0.58
2005-01-03,CO,GO,2.362289,1.591160,2.6682,2412.83,0.58
2005-01-03,CO,MS,2.542967,1.732000,2.6682,2412.83,0.58
2005-01-03,CO,MT,2.740508,1.767554,2.6682,2412.83,0.58
2005-01-03,N,AC,2.648780,1.990000,2.6682,2412.83,0.58
...,...,...,...,...,...,...,...
2021-12-31,NE,BA,6.750375,5.738000,5.5805,6120.04,0.73
2021-12-31,SE,ES,6.857250,6.146333,5.5805,6120.04,0.73
2021-12-31,SE,MG,6.817750,4.985167,5.5805,6120.04,0.73


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 70948 entries, 2005-01-03 to 2021-12-31
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Regiao              70948 non-null  object 
 1   Estado              70948 non-null  object 
 2   ValorVendaGasolina  70948 non-null  float64
 3   ValorVendaEtanol    70948 non-null  float64
 4   CotacaoDolarVenda   70948 non-null  float64
 5   Indice_Dez93        70948 non-null  float64
 6   Ipca_Mensal         70948 non-null  float64
dtypes: float64(5), object(2)
memory usage: 4.3+ MB


In [41]:
# Exporta o dataset para um arquivo CSV
df_combustiveis_dolar_ipca_estado.to_csv(pasta_datasets + 'df_combustiveis_dolar_ipca_estado.csv', sep = ';',index=True)

In [42]:
# Com o intuito de efetuar a análise de Machine Learning do dataset, a princípio será feita
# uma comparação utilizando o dataset com agrupamentos a nível nacional

In [43]:
df_combustiveis_agrupados_nacional = pd.DataFrame(df_combustiveis_agrupados_estado.groupby('Data')
                                                  [['ValorVendaGasolina','ValorVendaEtanol']].mean())

df_combustiveis_agrupados_nacional


,ValorVendaGasolina,ValorVendaEtanol
Data,,
2005-01-03,2.411366,1.750419
2005-01-04,2.406277,1.719137
2005-01-05,2.389795,1.740931
2005-01-06,2.424676,1.704239
2005-01-10,2.414220,1.744700
...,...,...
2021-12-27,6.668185,5.580209
2021-12-28,6.692192,5.629191
2021-12-29,6.726590,5.624709


In [44]:
# Exporta o dataset para um arquivo CSV
#df_combustiveis_agrupados_nacional.to_csv(pasta_datasets + 'df_combustiveis_agrupados_nacional.csv', sep = ';',index=True)

In [45]:
# Efetua o merge com o dataframe df_dolar_ipca

df_combustiveis_dolar_ipca_nacional = pd.merge(df_combustiveis_agrupados_nacional,df_dolar_ipca,on="Data",how="left")

display(df_combustiveis_dolar_ipca_nacional)


,ValorVendaGasolina,ValorVendaEtanol,CotacaoDolarVenda,Indice_Dez93,Ipca_Mensal
Data,,,,,
2005-01-03,2.411366,1.750419,2.6682,2412.83,0.58
2005-01-04,2.406277,1.719137,2.6887,2412.83,0.58
2005-01-05,2.389795,1.740931,2.7096,2412.83,0.58
2005-01-06,2.424676,1.704239,2.7207,2412.83,0.58
2005-01-10,2.414220,1.744700,2.6973,2412.83,0.58
...,...,...,...,...,...
2021-12-27,6.668185,5.580209,5.6650,6120.04,0.73
2021-12-28,6.692192,5.629191,5.6438,6120.04,0.73
2021-12-29,6.726590,5.624709,5.6619,6120.04,0.73


In [46]:
# Verifica a existência de valores null, sendo que:
# - as células ValorVendaGasolina e ValorVendaEtanol nulas são dias em que não houve coleta 
# - já para as colunas derivadas do dataframe df_dolar_ipca, verifica se que há 123 valores nulos,
# o que será verificado a seguir

df_combustiveis_dolar_ipca_nacional.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3634 entries, 2005-01-03 to 2021-12-31
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   ValorVendaGasolina  3634 non-null   float64
 1   ValorVendaEtanol    3634 non-null   float64
 2   CotacaoDolarVenda   3511 non-null   float64
 3   Indice_Dez93        3511 non-null   float64
 4   Ipca_Mensal         3511 non-null   float64
dtypes: float64(5)
memory usage: 170.3 KB


In [47]:
# Esses são os mesmos dias da análise anterior, que não são dias úteis e portanto não devem existir 
# no dataframe df_combustiveis_agrupados_nacional

datas_valores_nulos_dolar_distintos_nacional = df_combustiveis_dolar_ipca_nacional[df_combustiveis_dolar_ipca_nacional['CotacaoDolarVenda'].isnull()].index.unique()

datas_valores_nulos_dolar_distintos_nacional

DatetimeIndex(['2005-02-07', '2005-02-08', '2005-04-21', '2005-05-26',
               '2005-06-04', '2006-02-27', '2007-02-19', '2008-02-04',
               '2009-02-23', '2009-08-30',
               ...
               '2021-08-01', '2021-08-08', '2021-08-15', '2021-08-22',
               '2021-08-29', '2021-09-05', '2021-09-07', '2021-10-12',
               '2021-11-02', '2021-11-15'],
              dtype='datetime64[ns]', name='Data', length=123, freq=None)

In [48]:
# Assim será feita a remoção destas linhas que não são dias uteis

In [49]:
# ANTES

display(df_combustiveis_dolar_ipca_nacional)
df_combustiveis_dolar_ipca_nacional.info()

,ValorVendaGasolina,ValorVendaEtanol,CotacaoDolarVenda,Indice_Dez93,Ipca_Mensal
Data,,,,,
2005-01-03,2.411366,1.750419,2.6682,2412.83,0.58
2005-01-04,2.406277,1.719137,2.6887,2412.83,0.58
2005-01-05,2.389795,1.740931,2.7096,2412.83,0.58
2005-01-06,2.424676,1.704239,2.7207,2412.83,0.58
2005-01-10,2.414220,1.744700,2.6973,2412.83,0.58
...,...,...,...,...,...
2021-12-27,6.668185,5.580209,5.6650,6120.04,0.73
2021-12-28,6.692192,5.629191,5.6438,6120.04,0.73
2021-12-29,6.726590,5.624709,5.6619,6120.04,0.73


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3634 entries, 2005-01-03 to 2021-12-31
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   ValorVendaGasolina  3634 non-null   float64
 1   ValorVendaEtanol    3634 non-null   float64
 2   CotacaoDolarVenda   3511 non-null   float64
 3   Indice_Dez93        3511 non-null   float64
 4   Ipca_Mensal         3511 non-null   float64
dtypes: float64(5)
memory usage: 170.3 KB


In [50]:
# DEPOIS

df_combustiveis_dolar_ipca_nacional.dropna(subset = ['CotacaoDolarVenda'], inplace=True)

display(df_combustiveis_dolar_ipca_nacional)
df_combustiveis_dolar_ipca_nacional.info()

,ValorVendaGasolina,ValorVendaEtanol,CotacaoDolarVenda,Indice_Dez93,Ipca_Mensal
Data,,,,,
2005-01-03,2.411366,1.750419,2.6682,2412.83,0.58
2005-01-04,2.406277,1.719137,2.6887,2412.83,0.58
2005-01-05,2.389795,1.740931,2.7096,2412.83,0.58
2005-01-06,2.424676,1.704239,2.7207,2412.83,0.58
2005-01-10,2.414220,1.744700,2.6973,2412.83,0.58
...,...,...,...,...,...
2021-12-27,6.668185,5.580209,5.6650,6120.04,0.73
2021-12-28,6.692192,5.629191,5.6438,6120.04,0.73
2021-12-29,6.726590,5.624709,5.6619,6120.04,0.73


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3511 entries, 2005-01-03 to 2021-12-31
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   ValorVendaGasolina  3511 non-null   float64
 1   ValorVendaEtanol    3511 non-null   float64
 2   CotacaoDolarVenda   3511 non-null   float64
 3   Indice_Dez93        3511 non-null   float64
 4   Ipca_Mensal         3511 non-null   float64
dtypes: float64(5)
memory usage: 164.6 KB


In [51]:
# Verificando os campos nulos, constata-se que há um dia em que não consta o dado em ValorVendaGasolina

# df_combustiveis_dolar_ipca_nacional[df_combustiveis_dolar_ipca_nacional['ValorVendaGasolina'].isna()]

In [52]:
# Optou-se assim por eliminar esta linha, para que o dataframe não tenha nenhum valor nulo

# df_combustiveis_dolar_ipca_nacional.dropna(subset = ['ValorVendaGasolina'], inplace=True)

# display(df_combustiveis_dolar_ipca_nacional)
# df_combustiveis_dolar_ipca_nacional.info()

In [53]:
# Exporta o dataset para um arquivo CSV
df_combustiveis_dolar_ipca_nacional.to_csv(pasta_datasets + 'df_combustiveis_dolar_ipca_nacional.csv', sep = ';',index=True)